# SQL Movie-Rating Exercise

![](https://cinemaspotlightblog.files.wordpress.com/2020/01/untitled-1.png?w=1004&h=558&crop=1)

You've started a new movie-rating website, and you've been collecting data on reviewers' ratings of various movies. There's not much data yet, but you can still try out some interesting queries. Here's the schema:

- Movie ( mID, title, year, director )
English: There is a movie with ID number mID, a title, a release year, and a director.

- Reviewer ( rID, name )
English: The reviewer with ID number rID has a certain name.

- Rating ( rID, mID, stars, ratingDate )
English: The reviewer rID gave the movie mID a number of stars rating (1-5) on a certain ratingDate.


Your queries will run over a small data set conforming to the schema.


In [ ]:
import sqlite3
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PATH = '/content/drive/MyDrive/FTMLE | 2021.03 | Japan/Week_2/Dataset/rating.sql'

In [ ]:
conn = sqlite3.connect("movie.db")
cursor = conn.cursor()
sql_file = open(PATH)
sql_as_string = sql_file.read()
cursor.executescript(sql_as_string)

In [ ]:
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type = 'table'", conn)

,name
0,Movie
1,Reviewer
2,Rating


In [ ]:
pd.read_sql_query("SELECT * FROM Movie", conn)

,mID,title,year,director
0,101,Gone with the Wind,1939,Victor Fleming
1,102,Star Wars,1977,George Lucas
2,103,The Sound of Music,1965,Robert Wise
3,104,E.T.,1982,Steven Spielberg
4,105,Titanic,1997,James Cameron
5,106,Snow White,1937,None
6,107,Avatar,2009,James Cameron
7,108,Raiders of the Lost Ark,1981,Steven Spielberg


In [ ]:
pd.read_sql_query("SELECT * FROM Reviewer", conn)

,rID,name
0,201,Sarah Martinez
1,202,Daniel Lewis
2,203,Brittany Harris
3,204,Mike Anderson
4,205,Chris Jackson
5,206,Elizabeth Thomas
6,207,James Cameron
7,208,Ashley White


In [ ]:
pd.read_sql_query("SELECT * FROM Rating", conn)

,rID,mID,stars,ratingDate
0,201,101,2,2011-01-22
1,201,101,4,2011-01-27
2,202,106,4,None
3,203,103,2,2011-01-20
4,203,108,4,2011-01-12
5,203,108,2,2011-01-30
6,204,101,3,2011-01-09
7,205,103,3,2011-01-27
8,205,104,2,2011-01-22
9,205,108,4,None


## Question 1

Find the titles of all movies directed by Steven Spielberg.

In [ ]:
query = '''
SELECT title FROM Movie WHERE director = 'Steven Spielberg'

'''

pd.read_sql_query(query, conn)

,title
0,E.T.
1,Raiders of the Lost Ark


## Question 2

Find all years that have a movie that received a rating of 4 or 5, and sort them in increasing order.

In [ ]:
query = '''
SELECT DISTINCT M.year

FROM Movie M
    JOIN Rating RA on M.mID = RA.mID

WHERE RA.stars in (4,5)

ORDER BY M.year
'''

pd.read_sql_query(query, conn)

,year
0,1937
1,1939
2,1981
3,2009


## Question 3

Write a query to return the ratings data in a more readable format: reviewer name, movie title, stars, and ratingDate. Also, sort the data, first by reviewer name, then by movie title, and lastly by number of stars.

In [ ]:
query = '''
SELECT R.name,M.title,RA.stars,RA.ratingDate

FROM Rating RA
    JOIN Movie M ON RA.mID = M.mID 
    JOIN Reviewer R ON RA.rID = R.rID

ORDER BY R.name,M.title,RA.stars
'''

pd.read_sql_query(query, conn)

,name,title,stars,ratingDate
0,Ashley White,E.T.,3,2011-01-02
1,Brittany Harris,Raiders of the Lost Ark,2,2011-01-30
2,Brittany Harris,Raiders of the Lost Ark,4,2011-01-12
3,Brittany Harris,The Sound of Music,2,2011-01-20
4,Chris Jackson,E.T.,2,2011-01-22
5,Chris Jackson,Raiders of the Lost Ark,4,None
6,Chris Jackson,The Sound of Music,3,2011-01-27
7,Daniel Lewis,Snow White,4,None
8,Elizabeth Thomas,Avatar,3,2011-01-15
9,Elizabeth Thomas,Snow White,5,2011-01-19


## Question 4

For all cases where the same reviewer rated the same movie twice and gave it a higher rating the second time, return the reviewer's name and the title of the movie.


In [ ]:
query = '''

'''

pd.read_sql_query(query, conn)

,name,title
0,Sarah Martinez,Gone with the Wind


## Question 5

For each movie, return the title and the 'rating spread', that is, the difference between highest and lowest ratings given to that movie. Sort by rating spread from highest to lowest, then by movie title.

In [ ]:
query = '''

SELECT M.Title,(MAX(RA.stars) - MIN(RA.stars)) AS Spread
FROM Rating RA
    JOIN Movie M USING (mID)
GROUP BY M.mID
ORDER BY Spread DESC,M.Title ASC

'''

pd.read_sql_query(query, conn)

,title,Spread
0,Avatar,2
1,Gone with the Wind,2
2,Raiders of the Lost Ark,2
3,E.T.,1
4,Snow White,1
5,The Sound of Music,1


## Question 6


For any rating where the reviewer is the same as the director of the movie, return the reviewer name, movie title, and number of stars.


In [ ]:
query = '''
SELECT R.name,M.title,RA.stars
FROM Rating RA
    JOIN Reviewer R USING (rID) 
    JOIN Movie M USING (mID) 
WHERE M.director = R.name
'''

pd.read_sql_query(query, conn)

,name,title,stars
0,James Cameron,Avatar,5


## Question 7

Find the titles of all movies not reviewed by Chris Jackson.


In [ ]:
query='''
SELECT Title 
FROM Movie
WHERE mID NOT IN
    (
    SELECT RA.mID
    FROM Rating RA
        JOIN Reviewer R USING (rID)
    WHERE R.name = 'Chris Jackson'
    )
'''

pd.read_sql_query(query, conn)

,title
0,Gone with the Wind
1,Star Wars
2,Titanic
3,Snow White
4,Avatar


## Question 8

Some directors directed more than one movie. For all such directors, return the titles of all movies directed by them, along with the director name. Sort by director name, then movie title.

In [ ]:
query='''
    SELECT director
    FROM Movie
    GROUP BY director
    HAVING COUNT(*)>1
'''
pd.read_sql_query(query, conn)

,director
0,James Cameron
1,Steven Spielberg


In [ ]:
query = '''

SELECT title,director 
FROM Movie

WHERE director in 
    (
    SELECT director
    FROM Movie
    GROUP BY director
    HAVING COUNT(*)>1
    )

ORDER BY director,title
'''
pd.read_sql_query(query, conn)

,title,director
0,Avatar,James Cameron
1,Titanic,James Cameron
2,E.T.,Steven Spielberg
3,Raiders of the Lost Ark,Steven Spielberg


## Question 9

Find the movie(s) with the lowest average rating. Return the movie title(s) and average rating. (Hint: This query may be more difficult to write in SQLite than other systems; you might think of it as finding the lowest average rating and then choosing the movie(s) with that average rating.)

In [ ]:
query='''
SELECT MIN(average) 
FROM (
    SELECT AVG(stars) AS Average 
    FROM Rating 
    GROUP BY mID
    )
'''
pd.read_sql_query(query, conn)

,MIN(average)
0,2.5


In [ ]:
query = '''
WITH AVERAGE_TABLE AS
(
    SELECT AVG(stars) AS Average 
    FROM Rating 
    GROUP BY mID
)


SELECT title,AVG(stars)
FROM Rating 
    JOIN Movie USING (mID)
GROUP BY mID
HAVING AVG(stars) = (SELECT MIN(average) FROM AVERAGE_TABLE)


'''
pd.read_sql_query(query, conn)

,title,AVG(stars)
0,The Sound of Music,2.5
1,E.T.,2.5


## Question 10

For each director, return the director's name together with the title(s) of the movie(s) they directed that received the highest rating among all of their movies, and the value of that rating. Ignore movies whose director is NULL.

In [ ]:
query='''
    SELECT mID,max(stars) AS star_max
    FROM Rating 
        JOIN Movie USING (mID)
    GROUP BY mID
'''
pd.read_sql_query(query, conn)

,mID,star_max
0,101,4
1,103,3
2,104,3
3,106,5
4,107,5
5,108,4


In [ ]:
query='''
WITH HighestRating AS
(
    SELECT mID,max(stars) AS star_max
    FROM Rating 
        JOIN Movie USING (mID)
    GROUP BY mID
)
 
SELECT director,title,max(star_max) AS star_max
FROM HighestRating
    JOIN Movie USING (mID)
WHERE director IS NOT NULL
GROUP BY director
'''
pd.read_sql_query(query, conn)

,director,title,star_max
0,James Cameron,Avatar,5
1,Robert Wise,The Sound of Music,3
2,Steven Spielberg,Raiders of the Lost Ark,4
3,Victor Fleming,Gone with the Wind,4


## Challenge: question 11

Find the difference between the average rating of movies released before 1980 and the average rating of movies released after 1980. (Make sure to calculate the average rating for each movie, then the average of those averages for movies before 1980 and movies after. Don't just calculate the overall average rating before and after 1980.)

In [ ]:
query='''

WITH 
Temp AS (
  SELECT m.title as Title, avg(ra.stars) as avgStar, m.year as Year
  FROM Movie AS m
    JOIN Rating AS ra ON ra.mID = m.mID
  GROUP BY m.title
),

Temp1 AS (
  SELECT avg(avgStar) as X
  FROM Temp
  WHERE Year <= 1980
),

Temp2 AS (
  SELECT avg(avgStar) as Y
  FROM Temp
  WHERE Year > 1980
)

--SELECT (SELECT X FROM Temp1) - (SELECT Y FROM Temp2)
SELECT Temp1.X - Temp2.Y
FROM Temp1 JOIN Temp2 USING (Common)
'''

pd.read_sql_query(query, conn)

,Temp1.X - Temp2.Y
0,0.055556
